<p style="font-weight:bold;"> <span style="font-size: 36px"> Equality Comparers </span> </p>

In [0]:
#!import "../DataModel/DataStructure"

In [0]:
class RawVariableComparer: IEqualityComparer<RawVariable>
{
    private bool IgnoreValues;
    private RawVariableComparer(bool ignoreValues)
    {
        IgnoreValues = ignoreValues;
    }

    public bool Equals(RawVariable x, RawVariable y) =>
        x.AccidentYear == y.AccidentYear && x.AmountType == y.AmountType && x.DataNode == y.DataNode && x.AocType == y.AocType && 
        x.Novelty == y.Novelty && x.EstimateType == y.EstimateType && (IgnoreValues ? true : x.Values.SequenceEqual(y.Values)); // TODO: enable Precision

    public int GetHashCode(RawVariable v) => IgnoreValues  
                                            ? HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty) 
                                            : HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty, v.Values);

    public static RawVariableComparer Instance(bool ignoreValues = false)
    { 
        return new RawVariableComparer(ignoreValues);
    }
}

In [0]:
class IfrsVariableComparer: IEqualityComparer<IfrsVariable>
{
    private bool IgnoreValue;
    private IfrsVariableComparer(bool ignoreValue)
    {
        IgnoreValue = ignoreValue;
    }

    public bool Equals(IfrsVariable x, IfrsVariable y) =>
        x.AccidentYear == y.AccidentYear && x.AmountType == y.AmountType && x.DataNode == y.DataNode && x.AocType == y.AocType && 
        x.Novelty == y.Novelty && x.EstimateType == y.EstimateType && (IgnoreValue ? true : Math.Abs(x.Value - y.Value) < Precision); 

    public int GetHashCode(IfrsVariable v) => IgnoreValue  
                                            ? HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty) 
                                            : HashCode.Combine(v.AccidentYear, v.AmountType, v.DataNode, v.AocType, v.Novelty, v.Value.ToString("F5"));

    public static IfrsVariableComparer Instance(bool ignoreValue = false)
    { 
        return new IfrsVariableComparer(ignoreValue);
    }
}

In [0]:
using System;
using System.Collections.Generic;
using System.Diagnostics; 
using System.Linq;
using System.Linq.Expressions;
using System.Reflection;
using System.Threading;
//using Systemorph.Domain;
//using Systemorph.Utils.Reflection;
using static Systemorph.Vertex.Equality.IdentityPropertyExtensions;

In [0]:
class EqualityComparer<T> : IEqualityComparer<T>
{
    private static readonly System.Reflection.PropertyInfo[] IdentityProperties = typeof(T).GetIdentityProperties().ToArray();
    private static Func<T, T, bool> compiledEqualityFunction;

    private EqualityComparer() {
        compiledEqualityFunction = GetEqualityFunction();
    }

    public static readonly EqualityComparer<T> Instance = new EqualityComparer<T>();

    public bool Equals(T x, T y) => compiledEqualityFunction(x, y);
    public int GetHashCode(T obj) => 0;

    private static Func<T, T, bool> GetEqualityFunction()
    {
        var prm1 = Expression.Parameter(typeof(T));
        var prm2 = Expression.Parameter(typeof(T));

        // r1 == null && r2 == null
        var nullConditionExpression = Expression.AndAlso(Expression.Equal(prm1, Expression.Constant(null, typeof(T))), Expression.Equal(prm2, Expression.Constant(null, typeof(T))));
        // r1 != null && r2 != null
        var nonNullConditionExpression = Expression.AndAlso(Expression.NotEqual(prm1, Expression.Constant(null, typeof(T))), Expression.NotEqual(prm2, Expression.Constant(null, typeof(T))));
        // r1.prop1 == r2.prop1 && r1.prop2 == r2.prop2...... 
        var allPropertiesEqualExpression = IdentityProperties.Select(propertyInfo => Expression.Equal(Expression.Property(prm1, propertyInfo), Expression.Property(prm2, propertyInfo))).Aggregate(Expression.AndAlso);

        var equalityExpr = Expression.OrElse(nullConditionExpression, Expression.AndAlso(nonNullConditionExpression, allPropertiesEqualExpression));
        return Expression.Lambda<Func<T, T, bool>>(equalityExpr, prm1, prm2).Compile();
    }
}